# [PACKUS] 클러스터 별 장바구니 분석

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.set_option("display.max_columns", None)

In [2]:
user = pd.read_csv('2x2_user.csv')

In [3]:
user.head()

,아이디,누적주문건수,총 사용 적립금,총구매금액,추천인아이디,평생회원,회원구분_x,회원등급,회원추가항목_가입경로,PC_,주문서 쿠폰 할인금액,이용기간,최근접속,cluster
0,91559380,5.0,0.0,234500.0,0,0.0,개인,일반,없음,0.800000,5000.0,461,540,[0 1]
1,danbi531,2.0,0.0,29200.0,0,0.0,개인,일반,없음,1.000000,0.0,247,750,[1 1]
2,scsodamae,15.0,20264.0,1034400.0,0,0.0,사업자,사업자,없음,0.341463,10000.0,197,755,[1 0]
3,ehfdl1108,1.0,0.0,77000.0,0,0.0,사업자,사업자,없음,1.000000,5000.0,215,736,[1 0]
4,wanya,4.0,0.0,172000.0,0,0.0,개인,일반,없음,0.545455,3000.0,157,792,[1 1]


In [4]:
order = pd.read_excel('(통합)고객 주문리스트17-19.xlsx')

In [31]:
order2 = order.copy()

### 전처리 과정에서, 사은품 특징 날라갈까봐
- 사은품 & 비사은품 구분 후, 
- 비사은품 전처리
- 다시 사은품+비사은품 합치기

In [32]:
order2['주문상품명'].nunique()

667

In [33]:
# 사은품
present = order2[order2['주문상품명'].str.contains('사은품')==1]

In [34]:
# 비사은품
order2 = order2[order2['주문상품명'].str.contains('사은품')==0]

## 1. '주문상품명' text 전처리 

In [37]:
#order['주문상품명'].str.split(' 대 ')
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split(' 소 ')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split(' 중 ')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split(' 대 ')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split(' 특대 ')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split(' 왕대 ')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('0개')[0])

In [38]:
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(소)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(중)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(대)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(특대)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(왕대)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('0개')[0])

In [39]:
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('투명')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('검정')[0])

In [40]:
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('1호')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('2호')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('3호')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('4호')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('5호')[0])

In [41]:
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split(' 백색')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split(' 02 20')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split(' 04 20')[0])

In [42]:
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('블랙')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('화이트')[0])

In [43]:
# PP 1~6칸 도시락용기   
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('1칸')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('2칸')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('3칸')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('4칸')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('5칸')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('6칸')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('7칸')[0])

In [44]:
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('2개')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('3개')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('4개')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('100매')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('약 270ml')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('약 480ml')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('약720ml')[0])

In [45]:
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(1개입)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(1개)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(2개입)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(2개)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(3개입)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(3개)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(4개입)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(4개)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(5개입)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(5개)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(6개입)')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('(6개)')[0])

In [46]:
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('2-')[0])
order2['주문상품명'] = order2['주문상품명'].apply(lambda x: x.split('3-')[0])

In [47]:
order2.loc[order2['주문상품명'].str.contains('실링필름', case=False), '주문상품명'] = '실링필름'
order2.loc[order2['주문상품명'].str.contains('실링용기', case=False), '주문상품명'] = '실링용기'
order2.loc[order2['주문상품명'].str.contains('수세미', case=False), '주문상품명'] = '수세미'
order2.loc[order2['주문상품명'].str.contains('고무장갑', case=False), '주문상품명'] = '고무장갑'
order2.loc[order2['주문상품명'].str.contains('고무 장갑', case=False), '주문상품명'] = '고무장갑'
order2.loc[order2['주문상품명'].str.contains('반찬용기', case=False), '주문상품명'] = '반찬용기'
order2.loc[order2['주문상품명'].str.contains('반찬 용기', case=False), '주문상품명'] = '반찬 용기'
order2.loc[order2['주문상품명'].str.contains('지퍼 백', case=False), '주문상품명'] = '지퍼백'
order2.loc[order2['주문상품명'].str.contains('지퍼백', case=False), '주문상품명'] = '지퍼백'
order2.loc[order2['주문상품명'].str.contains('나무젓가락', case=False), '주문상품명'] = '나무젓가락'
order2.loc[order2['주문상품명'].str.contains('나무 젓가락', case=False), '주문상품명'] = '나무젓가락'
order2.loc[order2['주문상품명'].str.contains('아이스컵', case=False), '주문상품명'] = '아이스컵'
order2.loc[order2['주문상품명'].str.contains('수저', case=False), '주문상품명'] = '수저' # 175mm, 170mm 그게 그거
order2.loc[order2['주문상품명'].str.contains('92파이', case=False), '주문상품명'] = '플라스틱컵'
order2.loc[order2['주문상품명'].str.contains('98파이', case=False), '주문상품명'] = '플라스틱컵' # 92파이, 98파이는 플라스틱 일회용컵!

In [48]:
order2['주문상품명'] = order2['주문상품명'].str.strip()

In [49]:
order2['주문상품명'].nunique()

266

### (2) 사은품 전처리

In [50]:
present['주문상품명'].value_counts()

사은품) 반짝이 수세미              1577
사은품) 위생 장갑 50매            1572
사은품) 독일정품 칼라 행주(3개입)      1457
사은품) 맛있숟갈 숟가락 100개(검정)    1329
사은품) 고무장갑                 1203
사은품) 위생 장갑 150매            381
사은품) 위생 롤백 2호              235
사은품) 위생 랩 2호               120
사은품) 위생 청 수세미(3개)           62
사은품) 칼라 3종 위생 면행주           54
사은품) 친환경 종이호일 2호            53
사은품) 위생 롤백 5호               52
사은품) 위생 랩 4호                42
사은품) 친환경 종이호일 1호            25
160mm 스포키 검정 100개 사은품       13
사은품) 다용도 스탠드 지퍼백M 15매        8
사은품) 다용도 스탠드 지퍼백XL 15매       5
Name: 주문상품명, dtype: int64

In [51]:
present.loc[present['주문상품명'].str.contains('장갑', case=False), '주문상품명'] = '사은품) 위생 장갑' 
present.loc[present['주문상품명'].str.contains('지퍼백', case=False), '주문상품명'] = '사은품) 지퍼백' 
present.loc[present['주문상품명'].str.contains('수세미', case=False), '주문상품명'] = '사은품) 수세미' 
present.loc[present['주문상품명'].str.contains('롤백', case=False), '주문상품명'] = '사은품) 롤백' 
present.loc[present['주문상품명'].str.contains('위생 랩', case=False), '주문상품명'] = '사은품) 위생 랩' 
present.loc[present['주문상품명'].str.contains('종이호일', case=False), '주문상품명'] = '사은품) 종이호일' 
present.loc[present['주문상품명'].str.contains('행주', case=False), '주문상품명'] = '사은품) 행주' 
present.loc[present['주문상품명'].str.contains('맛있숟갈', case=False), '주문상품명'] = '사은품) 숟가락' 
present.loc[present['주문상품명'].str.contains('160mm 스포키 검정 100개 사은품', case=False), '주문상품명'] = '사은품) 스포키' 

In [52]:
present['주문상품명'].value_counts()

사은품) 위생 장갑    3156
사은품) 수세미      1639
사은품) 행주       1511
사은품) 숟가락      1329
사은품) 롤백        287
사은품) 위생 랩      162
사은품) 종이호일       78
사은품) 지퍼백        13
사은품) 스포키        13
Name: 주문상품명, dtype: int64

### 다시 사은품 + 비사은품

In [53]:
order3 = pd.concat([order2,present])

In [54]:
order3['주문상품명'] = order3['주문상품명'].replace('PP','PP 도시락용기')

In [55]:
# 개수 너무 적은 것은, 여기서 말고 개별 cluster에서 버리기
order3['주문상품명'].value_counts()

210파이 중화면용기                          4798
PP 도시락용기                             3469
사은품) 위생 장갑                           3156
195파이 탕용기                            2746
105파이 다용도컵 세트                        2533
실링용기                                 1861
사각 죽용기                               1812
사은품) 수세미                             1639
195파이 미니탕용기                          1629
사은품) 행주                              1511
225파이 탕용기                            1347
아이스컵                                 1339
사은품) 숟가락                             1329
70파이 다용도컵 세트                         1142
수저                                   1053
반찬용기                                  911
95파이 다용도컵 세트                          868
비닐봉투 맛난거들었쪙                           835
250파이 원형 접시 미니                        676
우동용기                                  582
실링필름                                  566
170파이 원형 샐러드                          484
비닐봉투 무지                               475
사각 찜용기                            

## 2. Cluster & ID 담겨있는 file 전처리

In [56]:
order3['결제일시(입금확인일)'] = pd.to_datetime(order3['결제일시(입금확인일)']).dt.to_period('D')
order3 = order3.set_index(order3['결제일시(입금확인일)'])
order3 = order3.dropna(subset=['주문자ID'])
order3_1 = order3.loc[order3['주문자ID'].str.contains("@",na=False)]
order3_2 = order3.loc[(order3['주문자ID'].str.contains("@",na=False))==False]
order3_1['주문자ID'] = order3_1['주문자ID'].str.extract(r'(\w+)@').dropna()
order4 = pd.concat([order3_1, order3_2])

C:\Users\samsung\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
C:\Users\samsung\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [57]:
order5 = order4[['주문자ID','주문상품명']]

In [58]:
order6 = pd.merge(order5,user[['아이디', 'cluster']],left_on='주문자ID',right_on='아이디')
order6.head()

,주문자ID,주문상품명,아이디,cluster
0,91559380,170파이 원형 샐러드,91559380,[0 1]
1,91559380,170파이 원형 샐러드,91559380,[0 1]
2,91559380,170파이 원형 샐러드,91559380,[0 1]
3,91559380,사은품) 위생 랩,91559380,[0 1]
4,91559380,사은품) 수세미,91559380,[0 1]


In [59]:
order6.cluster.unique()

array(['[0 1]', '[1 1]', '[0 0]', '[1 0]'], dtype=object)

In [60]:
c00 = order6[order6.cluster=='[0 0]']
c01 = order6[order6.cluster=='[0 1]']
c10 = order6[order6.cluster=='[1 0]']
c11 = order6[order6.cluster=='[1 1]']

In [61]:
c00.shape, c01.shape, c10.shape, c11.shape

((8149, 4), (9990, 4), (2125, 4), (3706, 4))

## 3. 각 Cluster당 주문 건수 10개 안되는 것 버림
- 더 엄격하게 기준 잡으려면, 뒷 단계 '장바구니 분석'할 때 하기!

In [62]:
counts = c00['주문상품명'].value_counts()
c00 = c00[~c00['주문상품명'].isin(counts[counts < 10].index)]

counts = c01['주문상품명'].value_counts()
c01 = c01[~c01['주문상품명'].isin(counts[counts < 10].index)]

counts = c10['주문상품명'].value_counts()
c10 = c10[~c10['주문상품명'].isin(counts[counts < 10].index)]

counts = c11['주문상품명'].value_counts()
c11 = c11[~c11['주문상품명'].isin(counts[counts < 10].index)]

In [63]:
c00.shape, c01.shape, c10.shape, c11.shape

((7958, 4), (9791, 4), (1906, 4), (3441, 4))

## 4. 장바구니 분석

In [64]:
C00 = c00.groupby('아이디')['주문상품명'].apply(list).values.tolist()
C01 = c01.groupby('아이디')['주문상품명'].apply(list).values.tolist()
C10 = c10.groupby('아이디')['주문상품명'].apply(list).values.tolist()
C11 = c11.groupby('아이디')['주문상품명'].apply(list).values.tolist()

In [65]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [66]:
def Basket(df,min_support=0.01):
    people_num = len(df)
    te = TransactionEncoder()
    te_ary = te.fit(df).transform(df)
    te.fit(df)
    itemdf = pd.DataFrame(te_ary, columns=te.columns_)
    result = apriori(itemdf, min_support=min_support,use_colnames=True)
    result['count(명)'] = result['support']*len(df)
    print('Cluster ',people_num,'명')
    return result.sort_values(by=['count(명)'],ascending=False)

In [67]:
def AR(basket,min_threshold):
    return association_rules(basket, metric="confidence", min_threshold=min_threshold)        

## 5. Result

#### Confidence
- Confidence (신뢰도) : 특정조건하에, 다른사건이얼마나자주발생?
- confidence(A->B) : A를산사람이B도샀을확률

#### Lift
- lift 쉽게생각해!
- 상품Y를구매한비율에비해, ‚X를구매한고객이‛Y를구매한비율이몇배?

In [68]:
Basket(C00)

Cluster  973 명


,support,itemsets,count(명)
41,0.264132,(사은품) 위생 장갑),257.0
0,0.235355,(105파이 다용도컵 세트),229.0
7,0.191161,(195파이 탕용기),186.0
38,0.173690,(사은품) 수세미),169.0
21,0.167523,(70파이 다용도컵 세트),163.0
24,0.167523,(PP 도시락용기),163.0
45,0.158273,(수저),154.0
10,0.151079,(210파이 중화면용기),147.0
43,0.139774,(사은품) 행주),136.0
39,0.124358,(사은품) 숟가락),121.0


In [69]:
AR(Basket(C00),0.5)

Cluster  973 명


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(사은품) 수세미),(사은품) 위생 장갑),0.173690,0.264132,0.095581,0.550296,2.083416,0.049704,1.636339
1,(사은품) 숟가락),(사은품) 위생 장갑),0.124358,0.264132,0.068859,0.553719,2.096376,0.036012,1.648890
2,(250파이 원형 접시 미니),(210파이 중화면용기),0.068859,0.151079,0.057554,0.835821,5.532338,0.047151,5.170700
3,(나무젓가락),(수저),0.072970,0.158273,0.056526,0.774648,4.894366,0.044977,3.735162
4,(실링필름),(실링용기),0.061665,0.095581,0.034943,0.566667,5.928674,0.029049,2.087122
5,(270파이 원형 접시),(210파이 중화면용기),0.039054,0.151079,0.032888,0.842105,5.573935,0.026988,5.376499
6,(300파이 원형 접시),(210파이 중화면용기),0.042138,0.151079,0.032888,0.780488,5.166086,0.026522,3.867306
7,"(105파이 다용도컵 세트, 사은품) 행주)",(사은품) 위생 장갑),0.044193,0.264132,0.030832,0.697674,2.641390,0.019160,2.434026
8,"(105파이 다용도컵 세트, 사은품) 수세미)",(사은품) 위생 장갑),0.051387,0.264132,0.030832,0.600000,2.271595,0.017259,1.839671
9,"(195파이 탕용기, 사은품) 수세미)",(사은품) 위생 장갑),0.044193,0.264132,0.029805,0.674419,2.553344,0.018132,2.260167


In [70]:
AR(Basket(C01),0.5)

Cluster  1426 명


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(사은품) 수세미),(사은품) 위생 장갑),0.190743,0.248948,0.097475,0.511029,2.052755,0.049990,1.535986
1,(사은품) 행주),(사은품) 위생 장갑),0.161992,0.248948,0.089060,0.549784,2.208426,0.048733,1.668202
2,"(사은품) 행주, 사은품) 수세미)",(사은품) 위생 장갑),0.064516,0.248948,0.041374,0.641304,2.576056,0.025313,2.093842
3,(사은품) 숟가락),(사은품) 위생 장갑),0.070827,0.248948,0.038569,0.544554,2.187422,0.020937,1.649049
4,(나무젓가락),(수저),0.045582,0.117812,0.031557,0.692308,5.876374,0.026187,2.867111
5,"(사은품) 위생 장갑, PP 도시락용기)",(사은품) 수세미),0.050491,0.190743,0.031557,0.625000,3.276654,0.021926,2.158018
6,"(105파이 다용도컵 세트, 사은품) 수세미)",(사은품) 위생 장갑),0.042777,0.248948,0.028050,0.655738,2.634034,0.017401,2.181627
7,(실링필름),(실링용기),0.040673,0.072931,0.027349,0.672414,9.219828,0.024383,2.829999
8,(300파이 원형 접시),(210파이 중화면용기),0.038569,0.114306,0.024544,0.636364,5.567206,0.020135,2.435659
9,(위생 랩),(사은품) 위생 장갑),0.036466,0.248948,0.024544,0.673077,2.703684,0.015466,2.297335


In [71]:
AR(Basket(C10),0.5)

Cluster  598 명


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(플라스틱컵),(아이스컵),0.065217,0.115385,0.051839,0.794872,6.888889,0.044314,4.312500
1,(실링필름),(실링용기),0.038462,0.073579,0.021739,0.565217,7.681818,0.018909,2.130769
2,(나무젓가락),(수저),0.018395,0.060201,0.015050,0.818182,13.590909,0.013943,5.168896
3,"(아이스컵, 사은품) 위생 장갑)",(플라스틱컵),0.020067,0.065217,0.013378,0.666667,10.222222,0.012069,2.804348
4,"(사은품) 위생 장갑, 플라스틱컵)",(아이스컵),0.016722,0.115385,0.013378,0.800000,6.933333,0.011448,4.423077
5,"(70파이 다용도컵 세트, PP 도시락용기)",(사은품) 위생 장갑),0.021739,0.240803,0.011706,0.538462,2.236111,0.006471,1.644928
6,(나무젓가락),(비닐봉투 맛난거들었쪙),0.018395,0.053512,0.010033,0.545455,10.193182,0.009049,2.082274
7,(250파이 원형 접시 미니),(210파이 중화면용기),0.016722,0.046823,0.010033,0.600000,12.814286,0.009250,2.382943


In [72]:
AR(Basket(C11),0.3)

Cluster  1287 명


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(아이스컵),(플라스틱컵),0.088578,0.044289,0.034188,0.385965,8.714681,0.030265,1.556444
1,(플라스틱컵),(아이스컵),0.044289,0.088578,0.034188,0.771930,8.714681,0.030265,3.996235
2,(사각 죽용기),(사은품) 위생 장갑),0.090909,0.219891,0.028749,0.316239,1.438163,0.008759,1.140909
3,(105파이 다용도컵 세트),(사은품) 위생 장갑),0.080031,0.219891,0.024864,0.310680,1.412879,0.007266,1.131707
4,(반찬용기),(사은품) 위생 장갑),0.052059,0.219891,0.022533,0.432836,1.968409,0.011086,1.375455
5,(실링용기),(사은품) 위생 장갑),0.054390,0.219891,0.020202,0.371429,1.689147,0.008242,1.241082
6,(수저),(사은품) 수세미),0.053613,0.135975,0.017094,0.318841,2.344845,0.009804,1.268462
7,(실링용기),(실링필름),0.054390,0.032634,0.016317,0.300000,9.192857,0.014542,1.381951
8,(실링필름),(실링용기),0.032634,0.054390,0.016317,0.500000,9.192857,0.014542,1.891220
9,(수세미),(PP 도시락용기),0.038850,0.142968,0.011655,0.300000,2.098370,0.006101,1.224331


## 6. Gephi ( Visualization )

In [46]:
listC00 = []
for i in range(len(C00)):
    listC00.append(list(set(C00[i])))
    
listC01 = []
for i in range(len(C01)):
    listC01.append(list(set(C01[i])))
    
listC10 = []
for i in range(len(C10)):
    listC10.append(list(set(C10[i])))
    
listC11 = []
for i in range(len(C11)):
    listC11.append(list(set(C11[i])))

In [64]:
pd.DataFrame(listC00).to_csv('Cluster[00].csv',header=False,index=False)
pd.DataFrame(listC01).to_csv('Cluster[01].csv',header=False,index=False)
pd.DataFrame(listC10).to_csv('Cluster[10].csv',header=False,index=False)
pd.DataFrame(listC11).to_csv('Cluster[11].csv',header=False,index=False)

### 위에서 생성된 4개의 csv파일로 Gephi 시각화!